In [1]:
import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import datetime
from datetime import datetime, timedelta
import time
import schedule
import numpy as np
import h5py

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Create directory
import os

path = './data'
# Check whether the specified path exists or not
isExist = os.path.exists(path)

if not isExist: 
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("new directory is created!")

In [3]:
today = datetime.now().date()
time_delta_D = timedelta(days=7)
lastweek = today - time_delta_D

today = str(today)
lastweek = str(lastweek)

today

'2022-02-20'

In [4]:
file_url = pd.read_csv('url.csv')
file_url

,url_list
0,https://finance.yahoo.com/cryptocurrencies/
1,https://finance.yahoo.com/gainers
2,https://finance.yahoo.com/currencies
3,https://coinmarketcap.com/nft/collections/


In [5]:
file_url['url_list'].iloc[0]

'https://finance.yahoo.com/cryptocurrencies/'

In [ ]:
def web_scrape(url):
    
    Bsoup = BeautifulSoup(requests.get(url).text)
    headers = [header.text for listing in Bsoup.find_all('thead') for header in listing.find_all('th')]
    r_data = {header:[] for header in headers}

    for rows in Bsoup.find_all('tbody'):
      for row in rows.find_all('tr'):
        
        if len(row) != len(headers): continue
        for idx, cell in enumerate(row.find_all('td')):
              r_data[headers[idx]].append(cell.text)
    return pd.DataFrame(r_data)


def job():

    df1 = web_scrape(file_url['url_list'].iloc[0])
    df = df1.iloc[:, 0:3]
    
    h5File = (today + '_d_web_scrape.h5')
    df.to_hdf(h5File, 'w')
    print("wrote hdf5 file done")   
    
schedule.every(1).minutes.do(job)
schedule.every(1).hour.do(job)
schedule.every(1).day.at("10:30").do(job)

In [ ]:
df1 = web_scrape(file_url['url_list'].iloc[0])
df = df1.iloc[:, 0:3]

h5File = (today + '_d_web_scrape.h5')
df.to_hdf(h5File, 'w')
print("wrote hdf5 file done")

In [ ]:
hf = pd.read_hdf(h5File)
hf.head(3)

In [ ]:
list(hf.keys())

In [ ]:
hf.shape

In [ ]:
%%writefile web_scraping.py

import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import datetime
from datetime import datetime, timedelta
import time
import schedule
import numpy as np
import h5py

import warnings
warnings.filterwarnings('ignore')

today = datetime.now().date()
today = str(today)

file_url = pd.read_csv('url.csv')

def web_scrape(url):
    
    Bsoup = BeautifulSoup(requests.get(url).text)
    headers = [header.text for listing in Bsoup.find_all('thead') for header in listing.find_all('th')]
    r_data = {header:[] for header in headers}

    for rows in Bsoup.find_all('tbody'):
      for row in rows.find_all('tr'):
        
        if len(row) != len(headers): continue
        for idx, cell in enumerate(row.find_all('td')):
              r_data[headers[idx]].append(cell.text)
    return pd.DataFrame(r_data)


def job():

    df1 = web_scrape(file_url['url_list'].iloc[0])
    df = df1.iloc[:, 0:3]
    
    h5File = (today + '_d_web_scrape.h5')
    df.to_hdf(h5File, 'w')
    print("wrote hdf5 file done")   
    
#schedule.every().day.at("10:30").do(job)

if __name__ == "__main__":
    job()